In [8]:
import pandas as pd
import numpy as np
import requests
import json

In [9]:
#base_url = "https://api.prod.bs-aws-stage.com/search/cases?addressTypes=villa,condo,terraced+house,holiday+house,cooperative,farm,hobby+farm,full+year+plot,villa+apartment,holiday+plot&per_page=50&page=1&sortAscending=true&sortBy=timeOnMarket"

In [10]:
n_max = 45134
per_page = 500
pages = [x for x in range(1,int(n_max/per_page)+1,1)]

In [11]:
urls = [f"https://api.prod.bs-aws-stage.com/search/cases?addressTypes=villa,condo,terraced+house,holiday+house,cooperative,farm,hobby+farm,full+year+plot,villa+apartment,holiday+plot&per_page={per_page}&page={x}&sortAscending=true&sortBy=timeOnMarket" for x in pages]

In [13]:
data = pd.DataFrame()
for url in urls:
    response = requests.get(url[:20]) # at urls[21] there is an error, so we only request for the first 10,000 observations
    print(url)
    data_500 = pd.DataFrame(response.json()["cases"])
    df_2 = [data, data_500]
    data = pd.concat(df_2, ignore_index=True)

ConnectionError: HTTPSConnectionPool(host='api.prod.bs-', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B2DC3551C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [7]:
data.head()

""


In [138]:
data.drop('_links', axis=1, inplace=True)

data.drop('caseID', axis=1, inplace=True)

data.drop('defaultImage', axis=1, inplace=True)

data.drop('images', axis=1, inplace=True)

data.drop('totalFavourites', axis=1, inplace=True)

data.drop('weightedArea', axis=1, inplace=True)

data.drop('distinction', axis=1, inplace=True)

data.drop('pageViews', axis=1, inplace=True)

data.drop('highlighted', axis=1, inplace=True)

data.drop('totalClickCount', axis=1, inplace=True)

data.drop('nextOpenHouse', axis=1, inplace=True)

data.drop('providerCaseID', axis=1, inplace=True)

data.drop('secondaryAddressType', axis=1, inplace=True)

data.drop('descriptionBody', axis=1, inplace=True)

data.drop('descriptionTitle', axis=1, inplace=True)

data.drop('slug', axis=1, inplace=True)

data.drop('status', axis=1, inplace=True)

data.drop('realtor', axis=1, inplace=True)

data.drop('cooperative', axis=1, inplace=True)

In [150]:
# extracting information on number of buildings from the "address" variable
list_numBuildings = []
for i in range(0,len(data),1):
    try:
        list_numBuildings.append(len(data["address"][i]["buildings"]))
    except:
        list_numBuildings.append(0)

data["numberOfBuildings"] = list_numBuildings

In [208]:
# if number of buildings is 0, it gets set to 1, so that e.g. 1 appartment counts as 1 building
for i in range(0,len(data),1):
    if (data["numberOfBuildings"].iloc[i] == 0):
        data["numberOfBuildings"].iloc[i] = 1

In [161]:
# Extracting Zip Code and Municipality Information from the Address Variable

data["zipCode"] = [data.address[i]["zipCode"] for i in range(0,len(data),1)]

data["municipality"] = [data["address"][i]["municipality"] for i in range(0,len(data),1)]

data["municipalityCode"] = [data["municipality"][i]["municipalityCode"] for i in range(0,len(data),1)]

data["municipality"] = [data["municipality"][i]["name"] for i in range(0,len(data),1)]



In [225]:
# for easy handling of the coordinates: two columns instead of one
data["latitude"] = [data["coordinates"][i]["lat"] for i in range(0,len(data),1)]
data["longitude"] = [data["coordinates"][i]["lon"] for i in range(0,len(data),1)]

KeyError: 'coordinates'

In [175]:
data.drop('coordinates', axis=1, inplace=True)

In [180]:
data.drop('address', axis=1, inplace=True)

In [181]:
data.drop('realEstate', axis=1, inplace=True)

### Basement and Lot
New indicator variables for Basement and Lot, that indicate whether or not there is any basement or any lot included in the real estate unit 

Nan are replaced with 0 based on the assumption that if the real estate agency does not report any basement/lot, then there is none.

In [197]:
basement_list = []
for i in range(0,len(data),1):
    if (data["basementArea"][i] > 0):
        basement_list.append(1)
    else:
        basement_list.append(0)
data["basement"] = basement_list

In [201]:
basement_list = []
for i in range(0,len(data),1):
    if (data["basementArea"][i] > 0):
        basement_list.append(data["basementArea"][i])
    else:
        basement_list.append(0)
data["basementArea"] = basement_list

In [221]:
lot_list = []
for i in range(0,len(data),1):
    if (data["lotArea"][i] > 0):
        lot_list.append(1)
    else:
        lot_list.append(0)
data["lot"] = lot_list

In [222]:
lot_list = []
for i in range(0,len(data),1):
    if (data["lotArea"][i] > 0):
        lot_list.append(data["lotArea"][i])
    else:
        lot_list.append(0)
data["lotArea"] = lot_list

In [202]:
data.columns

Index(['addressType', 'basementArea', 'caseUrl', 'energyLabel', 'housingArea',
       'lotArea', 'monthlyExpense', 'numberOfFloors', 'numberOfRooms',
       'perAreaPrice', 'priceCash', 'priceChangePercentage', 'yearBuilt',
       'municipalityCode', 'numberOfBuildings', 'zipCode', 'municipality',
       'latitude', 'longitude', 'basement'],
      dtype='object')

In [203]:
data.addressType.unique()

array(['villa', 'farm', 'holiday house', 'terraced house', 'cooperative',
       'condo', 'villa apartment', 'hobby farm', 'full year plot',
       'holiday plot'], dtype=object)

### There might be colinearity, for the variables lot, basement (and future indicator variables for addressType)

Theory: All units of type villa appartment have no basement and no lot

## Preview
Villas and Appartements in Copenhagen

In [226]:
data[((data["addressType"] == "villa") | (data["addressType"] == "villa apartment")) & (data["municipality"] == "København") ]

,addressType,basementArea,caseUrl,energyLabel,housingArea,lotArea,monthlyExpense,numberOfFloors,numberOfRooms,perAreaPrice,...,priceChangePercentage,yearBuilt,municipalityCode,numberOfBuildings,zipCode,municipality,latitude,longitude,basement,lot
291,villa,0.0,https://www.edc.dk/sag/?sagsnr=28902994,d,82.0,412.0,4656,1.0,4.0,62118.0,...,0.0,1956.0,101,2,2300,København,55.641243,12.597796,0,1
305,villa apartment,0.0,http://www.estate-maeglerne.dk/maegler/pages/p...,a2020,116.0,0.0,4078,1.0,4.0,46983.0,...,0.0,2022.0,101,1,2720,København,55.694206,12.472777,0,0
388,villa,0.0,https://www.danbolig.dk?propertyid=0630000142&...,c,101.0,511.0,4661,1.0,4.0,57376.0,...,0.0,1973.0,101,1,2700,København,55.707268,12.476728,0,1
400,villa,64.0,https://home.dk/sag/1450001308,f,134.0,728.0,7025,3.0,5.0,68848.0,...,0.0,1930.0,101,2,2300,København,55.649784,12.629133,1,1
416,villa,98.0,https://www.realmaeglerne.dk/301-redirect/?mgl...,c,186.0,649.0,5444,2.0,6.0,42163.0,...,0.0,1936.0,101,1,2300,København,55.649605,12.589301,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9007,villa,91.0,https://www.realmaeglerne.dk/301-redirect/?mgl...,c,98.0,697.0,4544,2.0,4.0,53825.0,...,0.0,1956.0,101,1,2500,København,55.653980,12.501422,1,1
9020,villa,0.0,https://home.dk/sag/1340000896,d,82.0,235.0,3051,1.0,3.0,42622.0,...,-6.8,1963.0,101,1,2300,København,55.644276,12.595235,0,1
9215,villa apartment,79.0,https://home.dk/sag/1340000029,d,94.0,464.0,2379,1.0,3.0,50840.0,...,0.0,1935.0,101,3,2300,København,55.647167,12.627104,1,1
9667,villa,91.0,https://home.dk/sag/1450001466,c,91.0,478.0,5743,2.0,4.0,58721.0,...,0.0,1934.0,101,2,2300,København,55.655758,12.631594,1,1


In [219]:
data.columns

Index(['addressType', 'basementArea', 'caseUrl', 'energyLabel', 'housingArea',
       'lotArea', 'monthlyExpense', 'numberOfFloors', 'numberOfRooms',
       'perAreaPrice', 'priceCash', 'priceChangePercentage', 'yearBuilt',
       'municipalityCode', 'numberOfBuildings', 'zipCode', 'municipality',
       'latitude', 'longitude', 'basement'],
      dtype='object')

In [227]:
# Save data to csv

In [228]:
#data.to_csv("data/name.csv")